In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import preprocessing


In [ ]:
df = pd.read_csv('./data/raw_data_cleaned.csv')

In [ ]:
df = df.rename(columns={'Unnamed: 0': 'Food Name'})

In [ ]:
df.columns

In [ ]:
for col in ['Plain Occurences', 'Cool Occurences', 'Warm Occurences',
       'Cold Occurences', 'Heavy Cold Occurences', 'Heavy Warm Occurences',
       'Hot Occurences', 'Heavy Hot Occurences']:
    print(df[col].value_counts().sort_values(ascending=False))

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
#df.describe()

In [ ]:
#df.info()

## Data Process

### Normalization

In [ ]:
df.columns

In [ ]:
def normalize(df):
    df.loc["Water (g)": "pH"] = (df.loc["Water (g)": "pH"]-df.loc["Water (g)": "pH"].mean())/df.loc["Water (g)": "pH"].std()
    
    name = df.iloc[:, 0:1]
    inputs = df.loc[:, 'Water (g)':'pH'].astype(float)
    outputs = df.loc[:, 'pH':].drop(columns='pH')

    # normalized_inputs = (inputs-inputs.mean())/inputs.std()
    normalized_inputs=(inputs-inputs.min())/(inputs.max()-inputs.min())
    
    df = pd.concat([name, normalized_inputs, outputs], axis=1)
    return df
    

In [ ]:
# new
def normalize_data(df):
    X = df.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)
    scaler = MinMaxScaler()
    scaler.fit(X)
    X_array = scaler.transform(X)
    new_data = pd.DataFrame(X_array,columns = X.columns)
    new_data['food_names'] = df['Unnamed: 0.1']
    
    return new_data

In [ ]:
df.loc["Water (g)": "pH"] = (df.loc["Water (g)": "pH"]-df.loc["Water (g)": "pH"].mean())/df.loc["Water (g)": "pH"].std()

In [ ]:
name = df.iloc[:, 0:1]
inputs = df.loc[:, 'Water (g)':'pH'].astype(float)
outputs = df.loc[:, 'pH':].drop(columns='pH')

# normalized_inputs = (inputs-inputs.mean())/inputs.std()
normalized_inputs=(inputs-inputs.min())/(inputs.max()-inputs.min())

In [ ]:
df = pd.concat([name, normalized_inputs, outputs], axis=1)

### Y Feature Engineering

In [ ]:
def occurrences(df):
    df["% Plain"] = df["Plain Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Cool"] = df["Cool Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Warm"] = df["Warm Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Cold"] = df["Cold Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Heavy Cold"] = df["Heavy Cold Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Heavy Warm"] = df["Heavy Warm Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Hot"] = df["Hot Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Heavy Hot"] = df["Heavy Hot Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    return df

In [ ]:
def linear_scale_v1(df):
    df["hot_cold_scale"] = (df["% Plain"] * (3.0/7.0)) + (df["% Cool"] * (2.0/7.0)) + (df["% Warm"] * (4.0/7.0)) + (df["% Cold"] * (1.0/7.0)) + (df["% Heavy Cold"] * (0.0/7.0)) + (df["% Heavy Warm"] * (5.0/7.0)) + (df["% Hot"] * (6.0/7.0)) + (df["% Heavy Hot"] * (7.0/7.0))

    return df

In [ ]:
def linear_scale(df):
#     df["hot_cold_scale"] = (df["% Plain"] * (3.0/7.0)) + (df["% Cool"] * (2.0/7.0)) + (df["% Warm"] * (4.0/7.0)) + (df["% Cold"] * (1.0/7.0)) + (df["% Heavy Cold"] * (0.0/7.0)) + (df["% Heavy Warm"] * (5.0/7.0)) + (df["% Hot"] * (6.0/7.0)) + (df["% Heavy Hot"] * (7.0/7.0))
    df["hot_cold_scale"] = (df["% Plain"] * (3.0/6.0)) + (df["% Cool"] * (2.0/6.0)) + (df["% Warm"] * (4.0/6.0)) + (df["% Cold"] * (1.0/6.0)) + (df["% Heavy Cold"] * (0.0/6.0)) + (df["% Heavy Warm"] * (4.5/6.0)) + (df["% Hot"] * (5.0/6.0)) + (df["% Heavy Hot"] * (6.0/6.0))

    return df

In [ ]:
df = occurrences(df)

In [ ]:
def mode_old(df):
    newDf = pd.DataFrame(df.iloc[:, 39:47].idxmax(axis=1).str.rstrip("Occurences"))
    df["Mode"] = newDf[0]
    return df

def mode(df):
    df['Plain'] = df['Plain Occurences'].astype(int)
    df['Cold'] = df['Cool Occurences'].astype(int) + df['Cold Occurences'].astype(int) + df['Heavy Cold Occurences'].astype(int)
    df['Hot'] = df['Warm Occurences'].astype(int) + df['Heavy Warm Occurences'].astype(int) + df['Hot Occurences'].astype(int) + df['Heavy Hot Occurences'].astype(int)
    newDf = pd.DataFrame(df.loc[:, ['Plain', 'Cold', 'Hot']].idxmax(axis=1))
    df["Mode"] = newDf[0]
    return df

In [ ]:
df = linear_scale(df)

In [ ]:
df = mode(df)

In [ ]:
df.head(20)

### NaN Values

Iterative Imputation
https://machinelearningmastery.com/iterative-imputation-for-missing-values-in-machine-learning/

In [ ]:
def nanValuesRid(df):
    not_too_many_null_cols = df.columns[df.isnull().mean() < 0.3]
    df = df[not_too_many_null_cols]
    
    return df

In [ ]:
df.info()

In [ ]:
def iterative_imputation(df):
    ii_imp = IterativeImputer(estimator=ExtraTreesRegressor(), max_iter=10, random_state=1121218)

    name = df.iloc[:, 0:1]
    inputs = df.loc[:, 'Water (g)':'pH']
    outputs = df.loc[:, 'pH':].drop(columns='pH')
    
    not_too_many_null_cols = df.columns[df.isnull().mean() < 0.3]
    
    inputs = ii_imp.fit_transform(inputs)
    new_inputs = pd.DataFrame(inputs, columns = not_too_many_null_cols[1:35]) 
    
    return pd.concat([name, new_inputs, outputs], axis=1)

In [ ]:
def label_encoder(df):
    # reading word labels
    label_encoder = preprocessing.LabelEncoder()

    # encode word labels in column
    df['hot_cold_scale'] = label_encoder.fit_transform(df['hot_cold_scale'])

    df['hot_cold_scale'].unique()

    return df
    

---

In [ ]:
df = pd.read_csv('./data/raw_data_cleaned.csv')

In [ ]:
df.head()

### Run All

In [ ]:
def data_process(df):
    df = normalize(df)
    df = occurrences(df)
    df = linear_scale(df)
    df = mode(df)
    
    df = iterative_imputation(df)
    
    df = label_encoder(df)
    
    return df


In [ ]:
df = data_process(df)

## Save Data

In [ ]:
df.info()

In [ ]:
#just keep adjust code and re-running and overwriting this file, to then use in pipeline to see results
df.to_csv('./data/data_processed.csv')